## Welcome to Skript.
This project is the work of Anand Chauhan, Vasu Jain, and Aayush Gupta. Students enrolled at Bennett University, for the purpose of a Probability and Statistics Hackathon as part of their curriculum.

In [193]:
# importing all libraries in one cell
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import re
import tensorflow as tf
import spacy
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

2022-11-06 02:11:59.188344: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-06 02:11:59.188497: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-06 02:11:59.188533: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-06 02:11:59.189638: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-11-06 02:11:59.189714: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1700] Could not ident

In [194]:
# import the sample dataset, created by Vasu Jain.
dataset = pd.read_csv('Speeches.csv')

In [195]:
# let the user select a speech and seperate every sentence from it.
f = open('sample.txt', 'r', encoding='UTF8')
sample = np.array(re.split(r'(\. )|(\n)', f.read()))
sample = sample[sample != np.array(None)]
cleansample = np.array([i for i in sample if len(i) > 6])

In [196]:
# load spacy NLP model
spacy.prefer_gpu()
nlp = spacy.load('en_core_web_lg')

In [197]:
# a sample of tokenizing with Spacy
doc = nlp(str(cleansample[1]))

print(f"{'text':{8}} {'POS':{6}} {'TAG':{6}} {'Dep':{6}} {'POS explained':{20}} {'tag explained'} ")
for token in doc:
    print(f'{token.text:{8}} {token.pos_:{6}} {token.tag_:{6}} {token.dep_:{6}} {spacy.explain(token.pos_):{20}} {spacy.explain(token.tag_)}')

text     POS    TAG    Dep    POS explained        tag explained 
Five     NUM    CD     nummod numeral              cardinal number
score    NOUN   NN     compound noun                 noun, singular or mass
years    NOUN   NNS    npadvmod noun                 noun, plural
ago      ADV    RB     advmod adverb               adverb
,        PUNCT  ,      punct  punctuation          punctuation mark, comma
a        DET    DT     det    determiner           determiner
great    ADJ    JJ     amod   adjective            adjective (English), other noun-modifier (Chinese)
American ADJ    JJ     nsubj  adjective            adjective (English), other noun-modifier (Chinese)
in       ADP    IN     prep   adposition           conjunction, subordinating or preposition
whose    DET    WP$    poss   determiner           wh-pronoun, possessive
symbolic ADJ    JJ     amod   adjective            adjective (English), other noun-modifier (Chinese)
shadow   NOUN   NN     pobj   noun                 noun, 

In [198]:
# a tangent, making a numpy array with every individual word.
words = [i.split() for i in cleansample]
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
cleanwords = []
for i in words:
    for x in i:
        x.replace(punc, "")
        cleanwords.append(x.lower())

In [199]:
# a simple script to detect persuasive sentences
results = dict(Counter(cleanwords))
modal = ['shall', 'should', 'will', 'would', 'must', 'ought', 'used', 'need', 'dare']
for i in modal:
    try:
        print(f"count for {i} is: {results[i]}")
    except:
        continue

count for shall is: 5
count for will is: 27
count for would is: 2
count for must is: 8


In [200]:
# tokenizing every word with the NLP model.
rawspeech = " ".join(cleansample.tolist())
proc = nlp(rawspeech)

In [201]:
# Experimental cell to find the best approach to find all 5 scores.
auxcount = 0
modcount = 0
for i in proc:
    if i.dep_ == 'aux':
        auxcount += 1
    if i.text in modal:
        modcount += 1

PERscore = np.sqrt(modcount/auxcount)
print(PERscore)

print(proc.similarity(nlp("progressive")))

0.6354889093022426
0.6973077654838562


In [202]:
# a script that cleans every speech in the dataset and adds it to the dataframe
# megasample and megawords contain the megaArray that can just be slotted in into the dataframe
megasample = []
megawords = []
# loop the cleaning script across all the speeches of the dataset.
# this code can also be used as general cleaning of input data.
for i in range(len(dataset)):
    cleansample = []
    cleanwords = []
    sample = np.array(re.split(r'(\. )|(\n)|(;)', dataset.loc[i, 'Speech']))
    sample = sample[sample != np.array(None)]
    cleansample = np.array([i for i in sample if len(i) > 4])
    words = [i.split() for i in cleansample]
    punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    for i in words:
        for x in i:
            x.replace(punc, "")
            cleanwords.append(x.lower())
    megasample.append(cleansample)
    megawords.append(cleanwords)
    
    
dataset['cleansample'] = megasample
dataset['cleanwords'] = megawords
dataset.head()

,Title,Date,Author,Speech,cleansample,cleanwords
0,Eighth Annual Message to Congress,7-12-1796,George Washington,Fellow Citizens of the Senate and House of Rep...,[Fellow Citizens of the Senate and House of Re...,"[fellow, citizens, of, the, senate, and, house..."
1,Declaration of Independence Anniversary Commem...,05-07-1926,Calvin Coolidge,Fellow Countrymen:\nWe meet to celebrate the b...,"[Fellow Countrymen:, We meet to celebrate the ...","[fellow, countrymen:, we, meet, to, celebrate,..."
2,Address on the Nuclear Test Ban Treaty,26-07-1963,John F. Kennedy,"Good evening, my fellow citizens:\nI speak to ...","[Good evening, my fellow citizens:, I speak to...","[good, evening,, my, fellow, citizens:, i, spe..."
3,Inaugural Address,20-01-2017,Donald Trump,"Chief Justice Roberts, President Carter, Presi...","[Chief Justice Roberts, President Carter, Pres...","[chief, justice, roberts,, president, carter,,..."
4,Gettysburg Address,19-11-1863,Abraham Lincoln,Four score and seven years ago our fathers bro...,[Four score and seven years ago our fathers br...,"[four, score, and, seven, years, ago, our, fat..."
